<a href="https://colab.research.google.com/github/minchillo4/factor-invest/blob/main/portproj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Análise Quantitativa de Ações B3 📊

O objetivo desse notebook é montar um modelo simplificado de investimento por fatores.


Primeiro, vamos instalar e importar as bibliotecas necessárias


In [ ]:
!pip install fundamentus
!pip install yfinance
!pip install vaderSentiment
!pip install googletrans==4.0.0-rc1
!pip install GoogleNews
!pip install newspaper3k
!pip install --upgrade deepl
!pip install translators --upgrade
!pip install finvader


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 8.6 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=620aff17022e536162616a31a162b0ca21fb25e4ad2583bd76cc50738cb

In [ ]:
import fundamentus
import pandas as pd
import numpy as np
import yfinance as yf
import sqlite3


/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


Agora, vamos extrair indicadores fundamentalistas de ações da B3.

In [ ]:
df_acoes = fundamentus.get_resultado_raw()
df_acoes

Multiples,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
papel,,,,,,,,,,,,,,,,,,,,
AALR3,8.44,-3.79,0.98,0.865,0.0000,0.363,-4.46,46.79,-0.92,82.26,12.69,0.0185,-0.2210,0.74,0.0091,-0.2592,1436770.0,1.015110e+09,1.04,0.0354
ABCB3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.1429,0.0,5.721810e+09,0.00,0.7094
ABCB4,23.22,6.73,0.96,0.000,0.0648,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.1429,13812300.0,5.721810e+09,0.00,0.7094
ABEV3,13.20,13.79,2.30,2.522,0.0553,1.508,56.10,12.23,-23.63,11.41,8.40,0.2063,0.1882,1.11,0.1650,0.1671,287967000.0,9.025490e+10,0.04,0.1323
ABYA3,4.91,-214.80,1.76,2.055,0.0000,0.527,1.98,19.96,-2.75,33.67,33.67,0.1029,-0.0096,2.09,0.0278,-0.0082,0.0,2.920600e+08,1.31,0.1641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WLMM4,26.90,9.27,1.46,0.463,0.0334,1.186,2.71,7.67,3.18,6.96,6.72,0.0603,0.0499,4.54,0.1834,0.1577,18851.6,6.698980e+08,0.01,0.2265
WMBY3,25.39,-19.30,2.87,0.836,0.0000,0.182,1.20,8.62,-1.50,25.02,25.02,0.0970,-0.0705,1.44,0.0239,-0.1486,0.0,2.124390e+08,6.51,-0.1448
WSON33,67.00,8.07,0.98,1.067,0.0000,0.400,13.41,2.42,-0.89,5.08,5.08,0.4414,0.1358,1.26,0.1836,0.1217,0.0,2.148530e+09,1.21,0.0523


In [ ]:
print(df_acoes.columns)

Index(['Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo', 'P/Cap.Giro',
       'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA', 'Mrg Ebit',
       'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses', 'Patrim. Líq',
       'Dív.Brut/ Patrim.', 'Cresc. Rec.5a'],
      dtype='object', name='Multiples')


In [ ]:
df_acoes.head(5)

Multiples,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
papel,,,,,,,,,,,,,,,,,,,,
AALR3,8.44,-3.79,0.98,0.865,0.0000,0.363,-4.46,46.79,-0.92,82.26,12.69,0.0185,-0.2210,0.74,0.0091,-0.2592,1436770.0,1.015110e+09,1.04,0.0354
ABCB3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.1429,0.0,5.721810e+09,0.00,0.7094
ABCB4,23.22,6.73,0.96,0.000,0.0648,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.1429,13812300.0,5.721810e+09,0.00,0.7094
ABEV3,13.20,13.79,2.30,2.522,0.0553,1.508,56.10,12.23,-23.63,11.41,8.40,0.2063,0.1882,1.11,0.1650,0.1671,287967000.0,9.025490e+10,0.04,0.1323
ABYA3,4.91,-214.80,1.76,2.055,0.0000,0.527,1.98,19.96,-2.75,33.67,33.67,0.1029,-0.0096,2.09,0.0278,-0.0082,0.0,2.920600e+08,1.31,0.1641


É necessário observar que a lista de ações da biblioteca fundamentus possuí ativos que atualmente não são parte  da B3. Por isso, vamos usar uma função (Criada por @TradingComDados) para filtrar somente as ações que estão sendo negociadas na data de hoje (24/01/2024)

In [ ]:
l_acoes = []

In [ ]:
from datetime import datetime, timedelta

def is_traded_recently(acao):
  try:
    stock = yf.Ticker(acao)
    historical_data = stock.history(period='1mo')

    last_trading_day = historical_data.index[-1].date()

    today = datetime.now().date()
    window_start = today - timedelta(days=5)
    return window_start <= last_trading_day <= today
  except Exception:
    return False


In [ ]:
df_acoes['ACAO'] = df_acoes.index + '.SA'
df_acoes

Multiples,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,...,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,ACAO
papel,,,,,,,,,,,,,,,,,,,,,
AALR3,8.44,-3.79,0.98,0.865,0.0000,0.363,-4.46,46.79,-0.92,82.26,...,0.0185,-0.2210,0.74,0.0091,-0.2592,1436770.0,1.015110e+09,1.04,0.0354,AALR3.SA
ABCB3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,...,0.0000,0.0000,0.00,0.0000,0.1429,0.0,5.721810e+09,0.00,0.7094,ABCB3.SA
ABCB4,23.22,6.73,0.96,0.000,0.0648,0.000,0.00,0.00,0.00,0.00,...,0.0000,0.0000,0.00,0.0000,0.1429,13812300.0,5.721810e+09,0.00,0.7094,ABCB4.SA
ABEV3,13.20,13.79,2.30,2.522,0.0553,1.508,56.10,12.23,-23.63,11.41,...,0.2063,0.1882,1.11,0.1650,0.1671,287967000.0,9.025490e+10,0.04,0.1323,ABEV3.SA
ABYA3,4.91,-214.80,1.76,2.055,0.0000,0.527,1.98,19.96,-2.75,33.67,...,0.1029,-0.0096,2.09,0.0278,-0.0082,0.0,2.920600e+08,1.31,0.1641,ABYA3.SA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WLMM4,26.90,9.27,1.46,0.463,0.0334,1.186,2.71,7.67,3.18,6.96,...,0.0603,0.0499,4.54,0.1834,0.1577,18851.6,6.698980e+08,0.01,0.2265,WLMM4.SA
WMBY3,25.39,-19.30,2.87,0.836,0.0000,0.182,1.20,8.62,-1.50,25.02,...,0.0970,-0.0705,1.44,0.0239,-0.1486,0.0,2.124390e+08,6.51,-0.1448,WMBY3.SA
WSON33,67.00,8.07,0.98,1.067,0.0000,0.400,13.41,2.42,-0.89,5.08,...,0.4414,0.1358,1.26,0.1836,0.1217,0.0,2.148530e+09,1.21,0.0523,WSON33.SA


In [ ]:
 df_acoes['NEGOCIADA'] = df_acoes['ACAO'].apply(is_traded_recently)

ERROR:yfinance:ABCB3.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:ABYA3.SA: No data found, symbol may be delisted
ERROR:yfinance:ACES3.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:ACES4.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:AEDU11.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:AEDU3.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:AELP3.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:AESL3.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:AESL4.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:AFLU3.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:AFLU5.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:AGEI3.SA: No data found, symbol may be delisted
ERROR:yfinance:AGEN33.SA: No data found, symbol may b

In [ ]:
df_acoes = df_acoes[df_acoes['NEGOCIADA'] == True]
df_acoes

Multiples,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,...,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,ACAO,NEGOCIADA
papel,,,,,,,,,,,,,,,,,,,,,
AALR3,8.44,-3.79,0.98,0.865,0.0000,0.363,-4.46,46.79,-0.92,82.26,...,-0.2210,0.74,0.0091,-0.2592,1.436770e+06,1.015110e+09,1.04,0.0354,AALR3.SA,True
ABCB4,23.22,6.73,0.96,0.000,0.0648,0.000,0.00,0.00,0.00,0.00,...,0.0000,0.00,0.0000,0.1429,1.381230e+07,5.721810e+09,0.00,0.7094,ABCB4.SA,True
ABEV3,13.20,13.79,2.30,2.522,0.0553,1.508,56.10,12.23,-23.63,11.41,...,0.1882,1.11,0.1650,0.1671,2.879670e+08,9.025490e+10,0.04,0.1323,ABEV3.SA,True
ADHM3,1.56,-4.09,-1.29,770.924,0.0000,105.126,-1.43,-5.69,-1.27,-5.69,...,-188.6060,0.00,5.2170,0.3147,2.064000e+03,-1.977600e+07,0.00,0.4348,ADHM3.SA,True
AERI3,0.79,-8.97,1.20,0.304,0.0000,0.257,1.20,3.33,-2.34,6.81,...,-0.0339,1.46,0.1074,-0.1342,5.499690e+06,8.150790e+08,2.17,0.1041,AERI3.SA,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WIZC3,6.59,10.84,2.29,1.004,0.0676,0.459,-18.09,2.30,-1.28,2.52,...,0.1781,0.90,0.2437,0.2118,3.448670e+06,4.592540e+08,0.77,0.1169,WIZC3.SA,True
WLMM3,22.53,7.76,1.22,0.388,0.0362,0.994,2.27,6.43,2.66,5.71,...,0.0499,4.54,0.1834,0.1577,1.599210e+03,6.698980e+08,0.01,0.2265,WLMM3.SA,True
WLMM4,26.90,9.27,1.46,0.463,0.0334,1.186,2.71,7.67,3.18,6.96,...,0.0499,4.54,0.1834,0.1577,1.885160e+04,6.698980e+08,0.01,0.2265,WLMM4.SA,True


In [ ]:
acao_column = df_acoes['ACAO']
df_f = df_acoes.drop(['Cotação', 'NEGOCIADA','ACAO','Patrim. Líq','Cresc. Rec.5a'], axis=1)
df_f

acao_column

papel
AALR3    AALR3.SA
ABCB4    ABCB4.SA
ABEV3    ABEV3.SA
ADHM3    ADHM3.SA
AERI3    AERI3.SA
           ...   
WIZC3    WIZC3.SA
WLMM3    WLMM3.SA
WLMM4    WLMM4.SA
YDUQ3    YDUQ3.SA
ZAMP3    ZAMP3.SA
Name: ACAO, Length: 474, dtype: object

Começaremos a organizar o nosso esquema de dados. Este dataframe será transformado em uma tabela no SQLITE.

In [ ]:
conn = sqlite3.connect(':memory:')
cur = conn.cursor()


In [ ]:
df_f.to_sql('stocks_fundamentals', conn, index=True, if_exists='replace', index_label='papel')

474

In [ ]:
df_from_sql = pd.read_sql('SELECT * FROM stocks_fundamentals', conn)

# Display the DataFrame
df_from_sql.head()

,papel,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Dív.Brut/ Patrim.
0,AALR3,-3.79,0.98,0.865,0.0000,0.363,-4.46,46.79,-0.92,82.26,12.69,0.0185,-0.2210,0.74,0.0091,-0.2592,1436770.0,1.04
1,ABCB4,6.73,0.96,0.000,0.0648,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.1429,13812300.0,0.00
2,ABEV3,13.79,2.30,2.522,0.0553,1.508,56.10,12.23,-23.63,11.41,8.40,0.2063,0.1882,1.11,0.1650,0.1671,287967000.0,0.04
3,ADHM3,-4.09,-1.29,770.924,0.0000,105.126,-1.43,-5.69,-1.27,-5.69,-5.81,-135.4850,-188.6060,0.00,5.2170,0.3147,2064.0,0.00
4,AERI3,-8.97,1.20,0.304,0.0000,0.257,1.20,3.33,-2.34,6.81,5.77,0.0912,-0.0339,1.46,0.1074,-0.1342,5499690.0,2.17


In [ ]:

# Connect to SQLite database


# Create a cursor object


# Execute PRAGMA table_info to get table information
cur.execute("PRAGMA table_info(stocks_fundamentals);")

# Fetch all rows from the result
table_info = cur.fetchall()
columns = ['ID', 'Column_Name', 'Data_Type', 'Not_Null', 'Default_Value', 'Primary_Key']
df_table_info = pd.DataFrame(table_info, columns=columns)

# Display the DataFrame
df_table_info

,ID,Column_Name,Data_Type,Not_Null,Default_Value,Primary_Key
0,0,papel,TEXT,0,None,0
1,1,P/L,REAL,0,None,0
2,2,P/VP,REAL,0,None,0
3,3,PSR,REAL,0,None,0
4,4,Div.Yield,REAL,0,None,0
5,5,P/Ativo,REAL,0,None,0
6,6,P/Cap.Giro,REAL,0,None,0
7,7,P/EBIT,REAL,0,None,0
8,8,P/Ativ Circ.Liq,REAL,0,None,0
9,9,EV/EBIT,REAL,0,None,0


In [ ]:
  from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler=MinMaxScaler(feature_range=(-1, 1))

In [ ]:
# Inverse transform to get back the original data
scaled_data = scaler.fit_transform(df_f)

scaled_data


array([[ 0.68362135, -0.97404769, -0.98061677, ...,  0.85498708,
        -0.99830079, -0.33915771],
       [ 0.69392442, -0.97406269, -0.98141827, ...,  0.86125781,
        -0.98366472, -0.38575269],
       [ 0.70083884, -0.97305789, -0.97908142, ...,  0.86163521,
        -0.65943232, -0.38396057],
       ...,
       [ 0.69641204, -0.97368776, -0.98098926, ...,  0.86148862,
        -0.9999777 , -0.38530466],
       [ 0.71915323, -0.97336533, -0.98029061, ...,  0.85993536,
        -0.91915499, -0.31182796],
       [ 0.67511055, -0.97397271, -0.98106987, ...,  0.85768033,
        -0.9852949 , -0.34677419]])

In [ ]:
df_acao = pd.DataFrame(acao_column, columns=['ACAO'])

In [ ]:
df_acao_scaled = pd.DataFrame(scaled_data, columns=df_f.columns)


In [ ]:
import pandas as pd

# Create a DataFrame with the 'ACAO' column
df_acao = pd.DataFrame(acao_column, columns=['ACAO'])

# Add the scaled data to the new DataFrame
df_acao_scaled = pd.DataFrame(scaled_data, columns=df_f.columns)

# Reset the index for both DataFrames
df_acao.reset_index(drop=True, inplace=True)
df_acao_scaled.reset_index(drop=True, inplace=True)

# Concatenate the two DataFrames along the columns axis
result_df = pd.concat([df_acao, df_acao_scaled], axis=1)

# Display the result DataFrame
print(result_df)


         ACAO       P/L      P/VP       PSR  Div.Yield   P/Ativo  P/Cap.Giro  \
0    AALR3.SA  0.683621 -0.974048 -0.980617  -1.000000 -0.993094    0.751507   
1    ABCB4.SA  0.693924 -0.974063 -0.981418  -0.999514 -1.000000    0.751733   
2    ABEV3.SA  0.700839 -0.973058 -0.979081  -0.999585 -0.971311    0.754585   
3    ADHM3.SA  0.683328 -0.975750 -0.267092  -1.000000  1.000000    0.751661   
4    AERI3.SA  0.678548 -0.973883 -0.981137  -1.000000 -0.995111    0.751794   
..        ...       ...       ...       ...        ...       ...         ...   
469  WIZC3.SA  0.697950 -0.973065 -0.980488  -0.999493 -0.991268    0.750814   
470  WLMM3.SA  0.694933 -0.973868 -0.981059  -0.999728 -0.981089    0.751849   
471  WLMM4.SA  0.696412 -0.973688 -0.980989  -0.999749 -0.977437    0.751871   
472  YDUQ3.SA  0.719153 -0.973365 -0.980291  -0.999896 -0.988129    0.752090   
473  ZAMP3.SA  0.675111 -0.973973 -0.981070  -1.000000 -0.993132    0.754480   

       P/EBIT  P/Ativ Circ.Liq   EV/EBI

In [ ]:
result_df.to_sql('stocks_fundamentals_norm', conn, index=True, if_exists='replace', index_label='papel')

474

vamos agora filtrar o nosso DF e tirar parte das ações.

In [ ]:
print(result_df.quantile(q=0.4, numeric_only=True))

P/L                  0.692969
P/VP                -0.974093
PSR                 -0.980949
Div.Yield           -0.999958
P/Ativo             -0.994003
P/Cap.Giro           0.751773
P/EBIT              -0.395604
P/Ativ Circ.Liq     -0.814149
EV/EBIT             -0.630171
EV/EBITDA           -0.533396
Mrg Ebit             0.977476
Mrg. Líq.            0.885427
Liq. Corr.          -0.937631
ROIC                 0.627802
ROE                  0.860140
Liq.2meses          -0.999811
Dív.Brut/ Patrim.   -0.377240
Name: 0.4, dtype: float64


In [ ]:

condition1 = (result_df['ROE'] >= 0.860140)
condition2 = (result_df['P/L'] >=  0.694064)
condition3 = (result_df['P/EBIT'] >= -0.3773)

selected_stocks = result_df[condition1 & condition2 & condition3]

print(selected_stocks)


         ACAO       P/L      P/VP       PSR  Div.Yield   P/Ativo  P/Cap.Giro  \
6    AFLT3.SA  0.705236 -0.973335 -0.975024  -0.999443 -0.969294    0.752574   
103  CEBR3.SA  0.694150 -0.973808 -0.977078  -0.998942 -0.982364    0.751841   
105  CEBR6.SA  0.695090 -0.973673 -0.976480  -0.998977 -0.979929    0.751855   
112  CEGR3.SA  0.743452 -0.962972 -0.977941  -0.999908 -0.906607    0.740270   
132  CPLE5.SA  0.723100 -0.972428 -0.978348  -0.999875 -0.975039    0.752593   
137  CRPG3.SA  0.704394 -0.973530 -0.979621  -0.999014 -0.976466    0.751849   
138  CRPG5.SA  0.702964 -0.973635 -0.979772  -0.998924 -0.978445    0.751840   
139  CRPG6.SA  0.702964 -0.973635 -0.979772  -0.998924 -0.978426    0.751840   
182  EMAE4.SA  0.710016 -0.973395 -0.978133  -0.999574 -0.980138    0.751872   
230  GRND3.SA  0.698949 -0.973463 -0.979085  -0.998469 -0.969808    0.751880   
251  ITSA3.SA  0.694610 -0.973815 -0.968603  -0.999585 -0.981013    0.752780   
252  ITSA4.SA  0.694669 -0.973808 -0.968

In [ ]:
selected_stocks

,ACAO,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Dív.Brut/ Patrim.
6,AFLT3.SA,0.705236,-0.973335,-0.975024,-0.999443,-0.969294,0.752574,-0.375534,-0.755421,-0.623972,-0.521480,0.980733,0.886089,-0.890909,0.631937,0.860675,-0.999996,-0.385753
103,CEBR3.SA,0.694150,-0.973808,-0.977078,-0.998942,-0.982364,0.751841,-0.373570,-0.810582,-0.627745,-0.529353,0.979363,0.886879,-0.664463,0.631527,0.861935,-0.999812,-0.385753
105,CEBR6.SA,0.695090,-0.973673,-0.976480,-0.998977,-0.979929,0.751855,-0.370074,-0.810222,-0.626443,-0.527180,0.979363,0.886879,-0.664463,0.631527,0.861935,-0.999761,-0.385753
112,CEGR3.SA,0.743452,-0.962972,-0.977941,-0.999908,-0.906607,0.740270,-0.370262,-0.827444,-0.621452,-0.516658,0.978694,0.885517,-0.948760,0.645741,0.863315,-1.000000,-0.345878
132,CPLE5.SA,0.723100,-0.972428,-0.978348,-0.999875,-0.975039,0.752593,-0.370302,-0.817908,-0.620429,-0.521163,0.978506,0.885564,-0.918457,0.630031,0.860372,-0.999996,-0.356631
137,CRPG3.SA,0.704394,-0.973530,-0.979621,-0.999014,-0.976466,0.751849,-0.289492,-0.809777,-0.595160,-0.512917,0.977286,0.885601,-0.735537,0.626185,0.860526,-1.000000,-0.385305
138,CRPG5.SA,0.702964,-0.973635,-0.979772,-0.998924,-0.978445,0.751840,-0.298704,-0.810065,-0.598579,-0.515146,0.977286,0.885601,-0.735537,0.626185,0.860526,-0.999873,-0.385305
139,CRPG6.SA,0.702964,-0.973635,-0.979772,-0.998924,-0.978426,0.751840,-0.298677,-0.810065,-0.598569,-0.515137,0.977286,0.885601,-0.735537,0.626185,0.860526,-0.999953,-0.385305
182,EMAE4.SA,0.710016,-0.973395,-0.978133,-0.999574,-0.980138,0.751872,0.579341,-0.722178,-0.346973,-0.376578,0.977111,0.885678,-0.610468,0.625034,0.860272,-0.999767,-0.385753
230,GRND3.SA,0.698949,-0.973463,-0.979085,-0.998469,-0.969808,0.751880,-0.376717,-0.809753,-0.625150,-0.526340,0.978474,0.885786,-0.609366,0.633720,0.861345,-0.991435,-0.384409


In [ ]:
filtro_selecionadas = selected_stocks[selected_stocks['ACAO'].str.endswith('3.SA')]
filtro_selecionadas

,ACAO,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Dív.Brut/ Patrim.
6,AFLT3.SA,0.705236,-0.973335,-0.975024,-0.999443,-0.969294,0.752574,-0.375534,-0.755421,-0.623972,-0.521480,0.980733,0.886089,-0.890909,0.631937,0.860675,-0.999996,-0.385753
103,CEBR3.SA,0.694150,-0.973808,-0.977078,-0.998942,-0.982364,0.751841,-0.373570,-0.810582,-0.627745,-0.529353,0.979363,0.886879,-0.664463,0.631527,0.861935,-0.999812,-0.385753
112,CEGR3.SA,0.743452,-0.962972,-0.977941,-0.999908,-0.906607,0.740270,-0.370262,-0.827444,-0.621452,-0.516658,0.978694,0.885517,-0.948760,0.645741,0.863315,-1.000000,-0.345878
137,CRPG3.SA,0.704394,-0.973530,-0.979621,-0.999014,-0.976466,0.751849,-0.289492,-0.809777,-0.595160,-0.512917,0.977286,0.885601,-0.735537,0.626185,0.860526,-1.000000,-0.385305
230,GRND3.SA,0.698949,-0.973463,-0.979085,-0.998469,-0.969808,0.751880,-0.376717,-0.809753,-0.625150,-0.526340,0.978474,0.885786,-0.609366,0.633720,0.861345,-0.991435,-0.384409
251,ITSA3.SA,0.694610,-0.973815,-0.968603,-0.999585,-0.981013,0.752780,-0.247117,-0.832764,-0.574557,-0.486603,0.978202,0.888939,-0.909091,0.625595,0.861729,-0.997338,-0.379480
267,LAND3.SA,0.698479,-0.973088,-0.966125,-0.999389,-0.967049,0.749461,-0.366698,-0.823072,-0.619735,-0.514904,0.983440,0.888056,-0.980716,0.630103,0.862122,-0.998665,-0.379032
364,RADL3.SA,0.728115,-0.968896,-0.980152,-0.999913,-0.955901,0.752364,-0.368863,-0.837027,-0.620673,-0.525118,0.977632,0.885458,-0.919008,0.635100,0.861971,-0.816566,-0.361559
406,SMFT3.SA,0.719094,-0.972451,-0.975002,-0.999864,-0.978863,0.752409,-0.353304,-0.816791,-0.614709,-0.527907,0.978956,0.885791,-0.919008,0.627960,0.860525,-0.934492,-0.352599
457,VIVA3.SA,0.708449,-0.971483,-0.977692,-0.999922,-0.948043,0.752096,-0.372494,-0.796386,-0.622560,-0.523085,0.978958,0.885739,-0.834160,0.636107,0.862211,-0.907231,-0.379032


In [ ]:
filtro_selecionadas.to_sql('stocks_fundamentals_norm', conn, index=True, if_exists='replace', index_label='papel')

12

In [ ]:
tickers_selecionados = filtro_selecionadas['ACAO']
tickers_selecionados

6      AFLT3.SA
103    CEBR3.SA
112    CEGR3.SA
137    CRPG3.SA
230    GRND3.SA
251    ITSA3.SA
267    LAND3.SA
364    RADL3.SA
406    SMFT3.SA
457    VIVA3.SA
462    VSTE3.SA
465    WEGE3.SA
Name: ACAO, dtype: object

In [ ]:
from pandas import DataFrame
from GoogleNews import GoogleNews
from datetime import date, timedelta
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
from googletrans import Translator
import deepl
import translators as ts
from newspaper import Article
from newspaper import Config
from finvader import finvader


now = date.today()
now = now.strftime('%m-%d-%Y')
yesterday = date.today() - timedelta(days = 1)
yesterday = yesterday.strftime('%m-%d-%Y')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Agora que possuimos os indicadores fundamentalistas vamos criar

In [ ]:



def translate_to_english(text):
    translated_text = ts.translate_text(query_text=text, from_language='pt', to_language='en')
    return translated_text

In [ ]:
def analyze_sentiment(title):
    scores = finvader(title, use_sentibignomics = True,use_henry = True, indicator = 'compound' )
    return scores



In [ ]:

def get_news(stock):
  try:
    now = date.today()
    yesterday = date.today() - timedelta(days=30)
    google_news = GoogleNews(lang='pt-br',region='BR',period='1d')
    google_news.search(f"{stock} stock")
    result = google_news.result()
    df = DataFrame(result)
    df['title_english'] = df['title'].apply(translate_to_english)

    df['score'] = df['title_english'].apply(analyze_sentiment)
    sentiment_score = df['score'].mean()
    dic = {'ACAO':stock,'sentiment_score':sentiment_score}
    return dic
  except Exception as E:
      print(f"News didnt find for {stock}")
      dic = {'ACAO':stock,'sentiment_score':0}
      return dic








In [ ]:
sentiments = []
import time as time
for stock in tickers_selecionados:
    sentiment = get_news(stock)
    sentiments.append(sentiment)
    print(f'Appended sentiment for:{stock}')

Appended sentiment for:AFLT3.SA
News didnt find for CEBR3.SA
Appended sentiment for:CEBR3.SA
News didnt find for CEGR3.SA
Appended sentiment for:CEGR3.SA
News didnt find for CRPG3.SA
Appended sentiment for:CRPG3.SA
Appended sentiment for:GRND3.SA
Appended sentiment for:ITSA3.SA
Appended sentiment for:LAND3.SA
News didnt find for RADL3.SA
Appended sentiment for:RADL3.SA
Appended sentiment for:SMFT3.SA
Appended sentiment for:VIVA3.SA
Appended sentiment for:VSTE3.SA
Appended sentiment for:WEGE3.SA


In [ ]:
sentiments

[{'ACAO': 'AFLT3.SA', 'sentiment_score': 0.0849},
 {'ACAO': 'CEBR3.SA', 'sentiment_score': 0},
 {'ACAO': 'CEGR3.SA', 'sentiment_score': 0},
 {'ACAO': 'CRPG3.SA', 'sentiment_score': 0},
 {'ACAO': 'GRND3.SA', 'sentiment_score': -0.21803999999999996},
 {'ACAO': 'ITSA3.SA', 'sentiment_score': 0.148},
 {'ACAO': 'LAND3.SA', 'sentiment_score': 0.2064},
 {'ACAO': 'RADL3.SA', 'sentiment_score': 0},
 {'ACAO': 'SMFT3.SA', 'sentiment_score': 0.036559999999999995},
 {'ACAO': 'VIVA3.SA', 'sentiment_score': 0.04694444444444444},
 {'ACAO': 'VSTE3.SA', 'sentiment_score': 0.05106666666666665},
 {'ACAO': 'WEGE3.SA', 'sentiment_score': 0.12473999999999999}]

In [ ]:
sentiments_df = pd.DataFrame.from_dict(sentiments)
sentiments_df



,ACAO,sentiment_score
0,AFLT3.SA,0.084900
1,CEBR3.SA,0.000000
2,CEGR3.SA,0.000000
3,CRPG3.SA,0.000000
4,GRND3.SA,-0.218040
5,ITSA3.SA,0.148000
6,LAND3.SA,0.206400
7,RADL3.SA,0.000000
8,SMFT3.SA,0.036560
9,VIVA3.SA,0.046944


In [ ]:
filtro_selecionadas

,ACAO,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Dív.Brut/ Patrim.
6,AFLT3.SA,0.705236,-0.973335,-0.975024,-0.999443,-0.969294,0.752574,-0.375534,-0.755421,-0.623972,-0.521480,0.980733,0.886089,-0.890909,0.631937,0.860675,-0.999996,-0.385753
103,CEBR3.SA,0.694150,-0.973808,-0.977078,-0.998942,-0.982364,0.751841,-0.373570,-0.810582,-0.627745,-0.529353,0.979363,0.886879,-0.664463,0.631527,0.861935,-0.999812,-0.385753
112,CEGR3.SA,0.743452,-0.962972,-0.977941,-0.999908,-0.906607,0.740270,-0.370262,-0.827444,-0.621452,-0.516658,0.978694,0.885517,-0.948760,0.645741,0.863315,-1.000000,-0.345878
137,CRPG3.SA,0.704394,-0.973530,-0.979621,-0.999014,-0.976466,0.751849,-0.289492,-0.809777,-0.595160,-0.512917,0.977286,0.885601,-0.735537,0.626185,0.860526,-1.000000,-0.385305
230,GRND3.SA,0.698949,-0.973463,-0.979085,-0.998469,-0.969808,0.751880,-0.376717,-0.809753,-0.625150,-0.526340,0.978474,0.885786,-0.609366,0.633720,0.861345,-0.991435,-0.384409
251,ITSA3.SA,0.694610,-0.973815,-0.968603,-0.999585,-0.981013,0.752780,-0.247117,-0.832764,-0.574557,-0.486603,0.978202,0.888939,-0.909091,0.625595,0.861729,-0.997338,-0.379480
267,LAND3.SA,0.698479,-0.973088,-0.966125,-0.999389,-0.967049,0.749461,-0.366698,-0.823072,-0.619735,-0.514904,0.983440,0.888056,-0.980716,0.630103,0.862122,-0.998665,-0.379032
364,RADL3.SA,0.728115,-0.968896,-0.980152,-0.999913,-0.955901,0.752364,-0.368863,-0.837027,-0.620673,-0.525118,0.977632,0.885458,-0.919008,0.635100,0.861971,-0.816566,-0.361559
406,SMFT3.SA,0.719094,-0.972451,-0.975002,-0.999864,-0.978863,0.752409,-0.353304,-0.816791,-0.614709,-0.527907,0.978956,0.885791,-0.919008,0.627960,0.860525,-0.934492,-0.352599
457,VIVA3.SA,0.708449,-0.971483,-0.977692,-0.999922,-0.948043,0.752096,-0.372494,-0.796386,-0.622560,-0.523085,0.978958,0.885739,-0.834160,0.636107,0.862211,-0.907231,-0.379032


In [ ]:
sentiments_df.to_sql('sentiment_stocks', conn, index=True, if_exists='replace', index_label='papel')

sentiments_from_sql = pd.read_sql('SELECT * FROM sentiment_stocks', conn)

# Display the DataFrame
sentiments_from_sql

,papel,ACAO,sentiment_score
0,0,AFLT3.SA,0.084900
1,1,CEBR3.SA,0.000000
2,2,CEGR3.SA,0.000000
3,3,CRPG3.SA,0.000000
4,4,GRND3.SA,-0.218040
5,5,ITSA3.SA,0.148000
6,6,LAND3.SA,0.206400
7,7,RADL3.SA,0.000000
8,8,SMFT3.SA,0.036560
9,9,VIVA3.SA,0.046944


In [ ]:
api_key = '6TQ6RM00MG3P7HN2'


In [ ]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29413 sha256=c2a56401f495b9e1790a1e632216ce2fe1afa8ec1c89e1b5c12b80ec419dd7c3
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
import requests
import pandas as pd
from ta import add_all_ta_features
from ta.utils import dropna

symbol = 'AFLT3.SA'

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}.SA&apikey={api_key}'
r = requests.get(url)
data = r.json()
data
# hue = pd.DataFrame(data['Weekly Time Series']).T
# hue
# df_ta = dropna(hue)
# df_ta
# df_ta['1. open'] = df_ta['1. open'].astype('float')
# df_ta['2. high'] = df_ta['2. high'].astype('float')
# df_ta['3. low'] = df_ta['3. low'].astype('float')
# df_ta['4. close'] = df_ta['4. close'].astype('float')
# df_ta['5. volume'] = df_ta['5. volume'].astype('float')


# sentiments_df.to_sql('stocks_historical', conn, index=True, if_exists='replace', index_label='papel')


{'Meta Data': {'1. Information': 'Weekly Prices (open, high, low, close) and Volumes',
  '2. Symbol': 'AFLT3.SA.SA',
  '3. Last Refreshed': '2024-01-26',
  '4. Time Zone': 'US/Eastern'},
 'Weekly Time Series': {'2024-01-26': {'1. open': '7.9900',
   '2. high': '8.1700',
   '3. low': '7.6000',
   '4. close': '8.0000',
   '5. volume': '3300'},
  '2024-01-19': {'1. open': '7.8700',
   '2. high': '8.0800',
   '3. low': '7.8700',
   '4. close': '8.0800',
   '5. volume': '600'},
  '2024-01-12': {'1. open': '8.4500',
   '2. high': '8.4500',
   '3. low': '7.8700',
   '4. close': '7.8700',
   '5. volume': '1500'},
  '2024-01-05': {'1. open': '8.1100',
   '2. high': '8.4700',
   '3. low': '7.8000',
   '4. close': '8.4500',
   '5. volume': '4000'},
  '2023-12-28': {'1. open': '8.6000',
   '2. high': '8.6000',
   '3. low': '8.5500',
   '4. close': '8.5900',
   '5. volume': '500'},
  '2023-12-22': {'1. open': '8.4200',
   '2. high': '8.6900',
   '3. low': '8.3000',
   '4. close': '8.3000',
   '5. v

In [ ]:
historical_dfs = []
for symbol in tickers_selecionados:
  try:
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}.SA&apikey={api_key}'
    r = requests.get(url)
    data = r.json()
    hue = pd.DataFrame(data['Weekly Time Series']).T
    hue
    df_ta = dropna(hue)
    df_ta
    df_ta['1. open'] = df_ta['1. open'].astype('float')
    df_ta['2. high'] = df_ta['2. high'].astype('float')
    df_ta['3. low'] = df_ta['3. low'].astype('float')
    df_ta['4. close'] = df_ta['4. close'].astype('float')
    df_ta['5. volume'] = df_ta['5. volume'].astype('float')
    df_ta['ACAO'] = symbol
    historical_dfs.append(df_ta)
  except Exception as E:
    print(f'error:{E}, {symbol}')

  historical_dfs


error:'Weekly Time Series', VSTE3.SA


In [ ]:
historical_prices = pd.concat(historical_dfs)
historical_prices

,1. open,2. high,3. low,4. close,5. volume,ACAO
2024-01-26,7.99,8.17,7.60,8.00,3300.0,AFLT3.SA
2024-01-19,7.87,8.08,7.87,8.08,600.0,AFLT3.SA
2024-01-12,8.45,8.45,7.87,7.87,1500.0,AFLT3.SA
2024-01-05,8.11,8.47,7.80,8.45,4000.0,AFLT3.SA
2023-12-28,8.60,8.60,8.55,8.59,500.0,AFLT3.SA
...,...,...,...,...,...,...
2005-06-17,8.00,8.00,8.00,8.00,0.0,WEGE3.SA
2005-06-10,8.50,8.50,7.50,7.50,8.0,WEGE3.SA
2005-02-28,7.80,7.80,7.80,7.80,0.0,WEGE3.SA
2005-02-24,7.80,7.80,7.80,7.80,3.0,WEGE3.SA


In [ ]:
historical_prices.to_sql('historical_prices', conn, index=True, if_exists='replace', index_label='papel')

5847

In [ ]:
df_ta = add_all_ta_features(
    df_ta, open="1. open", high="2. high", low="3. low", close="4. close", volume="5. volume", fillna=True)

df_ta

,1. open,2. high,3. low,4. close,5. volume,ACAO,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
2024-01-26,33.49,33.95,33.04,33.17,24825200.0,WEGE3.SA,-1.773229e+07,24825200.0,-0.714286,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,33.170000,0.000000,0.000000,0.000000
2024-01-19,34.67,34.79,32.63,33.48,25483400.0,WEGE3.SA,-2.315931e+07,50308600.0,-0.460345,7.899854e+06,...,0.074502,0.014900,0.059601,0.211088,0.042218,0.168871,33.308958,0.934579,0.930239,0.934579
2024-01-12,35.86,36.30,34.48,34.67,21947000.0,WEGE3.SA,-4.052397e+07,72255600.0,-0.560842,1.050229e+07,...,0.416640,0.095248,0.321392,-0.766760,-0.119578,-0.647182,33.912220,3.554361,3.492651,4.522159
2024-01-05,36.91,37.29,35.57,35.97,25162500.0,WEGE3.SA,-5.398298e+07,97418100.0,-0.554137,1.367500e+07,...,0.985693,0.273337,0.712356,-0.484566,-0.192576,-0.291991,34.825010,3.749639,3.681050,8.441363
2023-12-28,36.57,37.22,36.45,36.91,9440200.0,WEGE3.SA,-5.214398e+07,106858300.0,-0.487973,1.298911e+07,...,1.633744,0.545419,1.088326,-5.593663,-1.272793,-4.320870,35.788072,2.613289,2.579726,11.275249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005-06-17,8.00,8.00,8.00,8.00,0.0,WEGE3.SA,1.500639e+08,-428290159.0,0.237845,-3.847531e+03,...,-19.599841,-19.184753,-0.415088,-94.455568,-91.771163,-2.684405,8.993060,6.666667,6.453852,-75.881821
2005-06-10,8.50,8.50,7.50,7.50,8.0,WEGE3.SA,1.500638e+08,-428290167.0,0.066391,-3.298455e+03,...,-19.554113,-19.258625,-0.295488,-94.932404,-92.403411,-2.528993,8.920499,-6.250000,-6.453852,-77.389207
2005-02-28,7.80,7.80,7.80,7.80,0.0,WEGE3.SA,1.500638e+08,-428290167.0,0.014791,-2.827247e+03,...,-19.087383,-19.224377,0.136994,-95.368997,-92.996528,-2.372469,8.879448,4.000000,3.922071,-76.484775
2005-02-24,7.80,7.80,7.80,7.80,3.0,WEGE3.SA,1.500638e+08,-428290164.0,0.014829,-2.423355e+03,...,-18.542004,-19.087902,0.545898,-95.767608,-93.550744,-2.216864,8.718570,0.000000,0.000000,-76.484775


criar funcao de buscar noticia
criar funcao de analisar o sentimento
toma o ticker cmo parametro
ticker + stock na busca google
retorna a media das noticias e cria uma coluna nova no df inicial

In [ ]:
sentiments_from_sql = pd.read_sql('SELECT * FROM sentiment_stocks', conn)
historical_prices = pd.read_sql('SELECT * FROM historical_prices', conn)
fundamentals = pd.read_sql('SELECT * FROM stocks_fundamentals_norm', conn)

# Display the DataFrame
sentiments_from_sql

,papel,ACAO,sentiment_score
0,0,AFLT3.SA,0.084900
1,1,CEBR3.SA,0.000000
2,2,CEGR3.SA,0.000000
3,3,CRPG3.SA,0.000000
4,4,GRND3.SA,-0.218040
5,5,ITSA3.SA,0.148000
6,6,LAND3.SA,0.206400
7,7,RADL3.SA,0.000000
8,8,SMFT3.SA,0.036560
9,9,VIVA3.SA,0.046944


In [ ]:
historical_prices['ACAO'].unique()

array(['AFLT3.SA', 'CEBR3.SA', 'CEGR3.SA', 'CRPG3.SA', 'GRND3.SA',
       'ITSA3.SA', 'LAND3.SA', 'RADL3.SA', 'SMFT3.SA', 'VIVA3.SA',
       'WEGE3.SA'], dtype=object)

In [ ]:
fundamentals

,papel,ACAO,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Dív.Brut/ Patrim.
0,6,AFLT3.SA,0.705236,-0.973335,-0.975024,-0.999443,-0.969294,0.752574,-0.375534,-0.755421,-0.623972,-0.521480,0.980733,0.886089,-0.890909,0.631937,0.860675,-0.999996,-0.385753
1,103,CEBR3.SA,0.694150,-0.973808,-0.977078,-0.998942,-0.982364,0.751841,-0.373570,-0.810582,-0.627745,-0.529353,0.979363,0.886879,-0.664463,0.631527,0.861935,-0.999812,-0.385753
2,112,CEGR3.SA,0.743452,-0.962972,-0.977941,-0.999908,-0.906607,0.740270,-0.370262,-0.827444,-0.621452,-0.516658,0.978694,0.885517,-0.948760,0.645741,0.863315,-1.000000,-0.345878
3,137,CRPG3.SA,0.704394,-0.973530,-0.979621,-0.999014,-0.976466,0.751849,-0.289492,-0.809777,-0.595160,-0.512917,0.977286,0.885601,-0.735537,0.626185,0.860526,-1.000000,-0.385305
4,230,GRND3.SA,0.698949,-0.973463,-0.979085,-0.998469,-0.969808,0.751880,-0.376717,-0.809753,-0.625150,-0.526340,0.978474,0.885786,-0.609366,0.633720,0.861345,-0.991435,-0.384409
5,251,ITSA3.SA,0.694610,-0.973815,-0.968603,-0.999585,-0.981013,0.752780,-0.247117,-0.832764,-0.574557,-0.486603,0.978202,0.888939,-0.909091,0.625595,0.861729,-0.997338,-0.379480
6,267,LAND3.SA,0.698479,-0.973088,-0.966125,-0.999389,-0.967049,0.749461,-0.366698,-0.823072,-0.619735,-0.514904,0.983440,0.888056,-0.980716,0.630103,0.862122,-0.998665,-0.379032
7,364,RADL3.SA,0.728115,-0.968896,-0.980152,-0.999913,-0.955901,0.752364,-0.368863,-0.837027,-0.620673,-0.525118,0.977632,0.885458,-0.919008,0.635100,0.861971,-0.816566,-0.361559
8,406,SMFT3.SA,0.719094,-0.972451,-0.975002,-0.999864,-0.978863,0.752409,-0.353304,-0.816791,-0.614709,-0.527907,0.978956,0.885791,-0.919008,0.627960,0.860525,-0.934492,-0.352599
9,457,VIVA3.SA,0.708449,-0.971483,-0.977692,-0.999922,-0.948043,0.752096,-0.372494,-0.796386,-0.622560,-0.523085,0.978958,0.885739,-0.834160,0.636107,0.862211,-0.907231,-0.379032


In [ ]:
cur.execute("DELETE FROM stocks_fundamentals_norm WHERE ACAO='VSTE3.SA'")
conn.commit()

In [ ]:
fundamentals = pd.read_sql('SELECT * FROM stocks_fundamentals_norm', conn)
fundamentals

,papel,ACAO,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Dív.Brut/ Patrim.
0,6,AFLT3.SA,0.705236,-0.973335,-0.975024,-0.999443,-0.969294,0.752574,-0.375534,-0.755421,-0.623972,-0.521480,0.980733,0.886089,-0.890909,0.631937,0.860675,-0.999996,-0.385753
1,103,CEBR3.SA,0.694150,-0.973808,-0.977078,-0.998942,-0.982364,0.751841,-0.373570,-0.810582,-0.627745,-0.529353,0.979363,0.886879,-0.664463,0.631527,0.861935,-0.999812,-0.385753
2,112,CEGR3.SA,0.743452,-0.962972,-0.977941,-0.999908,-0.906607,0.740270,-0.370262,-0.827444,-0.621452,-0.516658,0.978694,0.885517,-0.948760,0.645741,0.863315,-1.000000,-0.345878
3,137,CRPG3.SA,0.704394,-0.973530,-0.979621,-0.999014,-0.976466,0.751849,-0.289492,-0.809777,-0.595160,-0.512917,0.977286,0.885601,-0.735537,0.626185,0.860526,-1.000000,-0.385305
4,230,GRND3.SA,0.698949,-0.973463,-0.979085,-0.998469,-0.969808,0.751880,-0.376717,-0.809753,-0.625150,-0.526340,0.978474,0.885786,-0.609366,0.633720,0.861345,-0.991435,-0.384409
5,251,ITSA3.SA,0.694610,-0.973815,-0.968603,-0.999585,-0.981013,0.752780,-0.247117,-0.832764,-0.574557,-0.486603,0.978202,0.888939,-0.909091,0.625595,0.861729,-0.997338,-0.379480
6,267,LAND3.SA,0.698479,-0.973088,-0.966125,-0.999389,-0.967049,0.749461,-0.366698,-0.823072,-0.619735,-0.514904,0.983440,0.888056,-0.980716,0.630103,0.862122,-0.998665,-0.379032
7,364,RADL3.SA,0.728115,-0.968896,-0.980152,-0.999913,-0.955901,0.752364,-0.368863,-0.837027,-0.620673,-0.525118,0.977632,0.885458,-0.919008,0.635100,0.861971,-0.816566,-0.361559
8,406,SMFT3.SA,0.719094,-0.972451,-0.975002,-0.999864,-0.978863,0.752409,-0.353304,-0.816791,-0.614709,-0.527907,0.978956,0.885791,-0.919008,0.627960,0.860525,-0.934492,-0.352599
9,457,VIVA3.SA,0.708449,-0.971483,-0.977692,-0.999922,-0.948043,0.752096,-0.372494,-0.796386,-0.622560,-0.523085,0.978958,0.885739,-0.834160,0.636107,0.862211,-0.907231,-0.379032


In [ ]:
cur.execute("DELETE FROM sentiment_stocks WHERE ACAO='VSTE3.SA'")
conn.commit()
sentiments_from_sql = pd.read_sql('SELECT * FROM sentiment_stocks', conn)
sentiments_from_sql

,papel,ACAO,sentiment_score
0,0,AFLT3.SA,0.084900
1,1,CEBR3.SA,0.000000
2,2,CEGR3.SA,0.000000
3,3,CRPG3.SA,0.000000
4,4,GRND3.SA,-0.218040
5,5,ITSA3.SA,0.148000
6,6,LAND3.SA,0.206400
7,7,RADL3.SA,0.000000
8,8,SMFT3.SA,0.036560
9,9,VIVA3.SA,0.046944
